In [ ]:
# split results_value
table = table.set_column(
    table.column_names.index("result_value"),
    "result_value",
    pa.compute.split_pattern(table["result_value"], "/"),
)

df = table.to_pandas().pivot(
    index=["subject_id", "seq_num", "chartdate"],
    columns="result_name",
    values="result_value",
)

DF = df.copy()

# cols = [
#     "Blood Pressure",
#     "Weight (Lbs)",
#     "BMI (kg/m2)",
#     "Height (Inches)",
#     "Blood Pressure Sitting",
#     "Blood Pressure Standing (1 min)",
#     "Blood Pressure Lying",
#     "Blood Pressure Standing (3 mins)",
#     "BMI",
#     "Weight",
#     "Blood Pressure Standing",
#     "eGFR",
#     "Height",
# ]

for col in df.columns:
    s = df.pop(col)
    m = s.isna()
    s.loc[m] = [[]] * m.sum()
    columns = (
        [f"{col} (systolic)", f"{col} (diastolic)"]
        if "blood pressure" in col.lower()
        else [col]
    )
    frame = pd.DataFrame(s.to_list(), columns=columns)
    for new_col in frame:
        df[new_col] = pd.to_numeric(
            frame.pop(new_col),
            errors="coerce",
            downcast=pa.float32(),
            dtype_backend="pyarrow",
        )